# 03 — Clustering Exploration

Interactive exploration of temporal clustering results:
- Silhouette analysis and method comparison
- Cluster center visualization
- Geographic patterns in cluster assignments
- Deep-dive into individual clusters

**Prerequisites:** Run `make cluster` first.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy.cluster.hierarchy import dendrogram

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
%matplotlib inline

import sys
sys.path.insert(0, str(Path.cwd().parent))
from src.config import DATA_PROCESSED, RESULTS_DIR, extract_borough

## 1. Load Results

In [ ]:
# Cluster assignments
dist_assign = pd.read_parquet(RESULTS_DIR / 'cluster_assignments_district.parquet')
sect_assign = pd.read_parquet(RESULTS_DIR / 'cluster_assignments_section.parquet')

# Metrics
metrics = pd.read_csv(RESULTS_DIR / 'cluster_metrics.csv')

# Feature matrices
X_dist = np.load(RESULTS_DIR / 'feature_matrix_district.npy')
ids_dist = pd.read_csv(RESULTS_DIR / 'feature_ids_district.csv').iloc[:, 0].tolist()

print(f"District assignments: {dist_assign.shape}")
print(f"Section assignments: {sect_assign.shape}")
print(f"\nMetrics:")
metrics

## 2. Method Comparison

In [ ]:
# Silhouette scores side by side
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(metrics))
bars = ax.bar(x, metrics['silhouette'], color=sns.color_palette('Set2', len(metrics)))

labels = [f"{row['method']}\n({row['level']})" for _, row in metrics.iterrows()]
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_ylabel('Silhouette Score')
ax.set_title('Clustering Method Comparison')

for bar, val in zip(bars, metrics['silhouette']):
    if not np.isnan(val):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{val:.3f}', ha='center', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Silhouette vs k curves
sil_files = list(RESULTS_DIR.glob('silhouette_district_*.csv'))

fig, ax = plt.subplots(figsize=(10, 5))
for f in sil_files:
    data = pd.read_csv(f)
    method = data['method'].iloc[0] if 'method' in data.columns else f.stem
    ax.plot(data['k'], data['silhouette'], marker='o', markersize=5, label=method)

ax.set_xlabel('k (Number of Clusters)')
ax.set_ylabel('Silhouette Score')
ax.set_title('Silhouette Score vs k — District Level')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Cluster Centers & Members

In [ ]:
# K-Means cluster centers with member trajectories
labels = dist_assign['cluster_kmeans'].values
n_clusters = len(set(labels))

n_cols = min(3, n_clusters)
n_rows = (n_clusters + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(6 * n_cols, 4 * n_rows))
axes = np.atleast_2d(axes)

for k in range(n_clusters):
    row, col = divmod(k, n_cols)
    ax = axes[row, col]
    mask = labels == k
    members = X_dist[mask]

    for ts in members:
        ax.plot(ts, color='steelblue', alpha=0.2, linewidth=0.5)
    ax.plot(members.mean(axis=0), color='darkred', linewidth=2.5)
    
    member_names = [ids_dist[i] for i in np.where(mask)[0]]
    ax.set_title(f'Cluster {k} (n={mask.sum()})')
    ax.set_xlabel('Week')
    ax.set_ylabel('Z-score')

for k in range(n_clusters, n_rows * n_cols):
    row, col = divmod(k, n_cols)
    axes[row, col].set_visible(False)

plt.suptitle('K-Means Clusters — District Level', y=1.01, fontsize=14)
plt.tight_layout()
plt.show()

## 4. Cluster Membership

In [ ]:
# Which districts are in each cluster?
dist_assign['borough'] = dist_assign['District_Code'].apply(extract_borough)

for k in sorted(dist_assign['cluster_kmeans'].unique()):
    members = dist_assign[dist_assign['cluster_kmeans'] == k]
    print(f"\nCluster {k} ({len(members)} districts):")
    for borough in sorted(members['borough'].unique()):
        codes = members[members['borough'] == borough]['District_Code'].tolist()
        print(f"  {borough}: {', '.join(codes)}")

In [ ]:
# Cross-tabulation: cluster vs borough
ct = pd.crosstab(dist_assign['cluster_kmeans'], dist_assign['borough'])

fig, ax = plt.subplots(figsize=(10, 6))
ct.plot(kind='bar', stacked=True, ax=ax)
ax.set_xlabel('Cluster')
ax.set_ylabel('Number of Districts')
ax.set_title('Borough Composition of Each Cluster')
ax.legend(title='Borough')
plt.tight_layout()
plt.show()

## 5. Dendrogram

In [ ]:
linkage_path = RESULTS_DIR / 'linkage_district.npy'
if linkage_path.exists():
    Z = np.load(linkage_path)
    
    fig, ax = plt.subplots(figsize=(16, 8))
    dendrogram(Z, labels=ids_dist, leaf_rotation=90, leaf_font_size=8, ax=ax)
    ax.set_title('Hierarchical Clustering Dendrogram — District Level')
    ax.set_ylabel('Ward Distance')
    plt.tight_layout()
    plt.show()
else:
    print('No linkage matrix found — run clustering first')

## 6. Method Agreement

In [ ]:
# How often do methods agree on grouping?
from sklearn.metrics import adjusted_rand_score

cluster_cols = [c for c in dist_assign.columns if c.startswith('cluster_')]
ari_matrix = pd.DataFrame(index=cluster_cols, columns=cluster_cols, dtype=float)

for i, c1 in enumerate(cluster_cols):
    for j, c2 in enumerate(cluster_cols):
        mask = (dist_assign[c1] != -1) & (dist_assign[c2] != -1)  # exclude noise
        if mask.sum() > 0:
            ari_matrix.loc[c1, c2] = adjusted_rand_score(
                dist_assign.loc[mask, c1], dist_assign.loc[mask, c2]
            )

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(ari_matrix.astype(float), annot=True, cmap='YlOrRd', vmin=0, vmax=1, ax=ax)
ax.set_title('Adjusted Rand Index — Method Agreement')
plt.tight_layout()
plt.show()